In [1]:
import jsonlines
import os
import numpy as np
import pandas as pd
from utils import read_data, extract_data, partition
from evaluation import calculate_score, process_CTC

In [2]:
task_list=('CMeIE', 'CHIP-CDN', 'CHIP-CDEE', 'CHIP-MDCFNPC',
           'CHIP-CTC', 'KUAKE-QIC',
           'IMCS-V2-MRG', 'MedDG',)
pred_path = "pred"
true_path = "true"

In [3]:
target_list = ["test_predications.json"]

In [ ]:
score_dict = {target: [] for target in target_list}
label_dict = {}

for target in target_list:
    target_path = os.path.join(pred_path, target)
    
    if not os.path.exists(os.path.join(target_path, task_list[0])):  # needs partition
        # all_data = read_data(os.path.join(target_path, "test_predictions.json"))
        all_data = read_data(target_path)
        partition(extract_data(all_data), task_list, target_path)
    
    for task in task_list:

        pp = os.path.join(target_path, task)
        tp = os.path.join(true_path, task)
        pp = os.path.join(pp, "test_predictions.json")
        tp = os.path.join(tp, "test.json")

        if task == "CHIP-CTC":  # CTC needs post process
            post_process_function = process_CTC
        else:
            post_process_function = None

        score, labels, _ = calculate_score(task, pp, tp, post_process_function)
        score_dict[target].append(score)
        label_dict[task] = labels

In [ ]:
#上面的代码只把test_prediction拆开放在各个任务中，没有把prediction放进去
target_path = os.path.join(pred_path, "test.json")   #pred/test.json

all_data = read_data(target_path)
partition(extract_data(all_data), task_list, pred_path)

In [5]:
res_data, res_key = [], []
for key, value in score_dict.items():
    res_data.append(value)
    res_key.append(key)

res_df = pd.DataFrame(columns=task_list,
                      index=res_key,
                      data=res_data)
res_df["average"] = res_df.mean(axis=1)

In [ ]:
res_df.head(20)

In [ ]:
try:
    new_res_df = res_df.drop(columns=["CHIP-STS", "KUAKE-IR", "average"])
except:
    new_res_df = res_df
new_res_df["average"] = new_res_df.mean(axis=1)
new_res_df.head(50)

In [ ]:
for st in ["CHIP-CDN", "CHIP-MDCFNPC", "IMCS-V2-MRG", "KUAKE-QIC"]: 
    score, _, _ = calculate_score(st, 
                                  "pred/moelora/%s/test_predictions.json" % st, 
                                  "true/%s/dev.json" %st, 
                                  post_process_function)
    print("The score for task %s is: %.5f" % (st, score))